In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CyclicLR
from radam import RAdam, PlainRAdam, AdamW

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 64 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [5]:
def main():
    
    use_cuda = False
    base_lr = 0.001
    max_lr = 0.8
    epochs = 10
    device = "cpu"
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)
    
    model = Net().to(device)
    
#     optimizer = optim.Adam(model.parameters(), lr=base_lr)
    optimizer = torch.optim.SGD(model.parameters(), lr=base_lr, momentum=0.9)
    
    
    
    scheduler = CyclicLR(optimizer, base_lr, max_lr)
    
    for epoch in range(1, epochs+1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()
        
        torch.save(model.state_dict(), "../models/mnist_cnn.pt")

In [7]:
if __name__ == "__main__":
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300616
Train Epoch: 1 [4096/60000 (7%)]	Loss: 1.467576
Train Epoch: 1 [8192/60000 (14%)]	Loss: 0.627057
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.547084
Train Epoch: 1 [16384/60000 (27%)]	Loss: 0.336317
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.546575
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.400578
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.367106
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.481471
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.416188
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.450319
Train Epoch: 1 [45056/60000 (75%)]	Loss: 0.323821
Train Epoch: 1 [49152/60000 (82%)]	Loss: 0.258997
Train Epoch: 1 [53248/60000 (89%)]	Loss: 0.325254
Train Epoch: 1 [57344/60000 (96%)]	Loss: 0.285719

Test set: Average loss: 0.0189, Accuracy: 9410/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.459537
Train Epoch: 2 [4096/60000 (7%)]	Loss: 0.218658
Train Epoch: 2 [8192/60000 (14%)]	Loss: 0.267478
Train Epoch: 2 [12288/60000 (20%)]	Loss: 0.362008
Train